In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as mticker
import seaborn as sns
import math
import scipy
from scipy import stats
import datatable as dt
import os
import glob
from tqdm.auto import tqdm # library for progress bars
from IPython.display import clear_output # Library for live update of progress

# Data locations:

In [ ]:
folder_AIRS_20x20 = '.\\data\\AIRS\\L1B_globe_20x20\\'
folder_LBL_20x20 = '.\\data\\LBL\\lgridd_v4\\'
folder_LBL_cfc = '.\\data\\LBL\\cfc\\'

AIRS_ir = "AIRS.2021.04.09.121.L1B.AIRS_Rad.v5.0.25.0.G21100105112.hdf"
AIRS_vis = "AIRS.2021.04.09.121.L1B.VIS_Rad.v5.0.0.0.G21100104903.hdf"

wn = pd.read_csv("AIRS_wavenumbers.csv", usecols=['wavenumber'])
wn = wn.round(3)

HITRAN2020 = pd.read_csv("HITRAN2020_CO2_620_720.txt", delim_whitespace=True)
HITRAN2020 = HITRAN2020[HITRAN2020.columns[:2]]
HITRAN2020=HITRAN2020.rename(columns = {'[cm-1]':'Absorbance'})
# Prepare easier plotting by moving exponent to Y-label
HITRAN2020['Absorbance'] = HITRAN2020['Absorbance'] * 1E19

# Import data:

In [ ]:
# AIRS:
rgrid = pd.DataFrame()
print('Importing AIRS data...')
for file in glob.glob(folder_AIRS_20x20+'*.csv.gz'):
    rgrid = pd.concat([rgrid, pd.read_csv(file)])

rgrid = rgrid.astype({'year': 'int32'})
rgrid = rgrid.astype({'month': 'int32'})
rgrid = rgrid.drop(['scanang', 'sci', 'sza', 'state'], axis=1).set_index(['year', 'month', 'lat', 'lon']).sort_index(ascending=True)
print('Relabeling columns...')
rgrid = pd.concat([rgrid.iloc[:, :1864], rgrid.iloc[:, -1]], axis=1) # drop wavenumbers higher than 1613.862
rgrid.columns = np.append([[wn['wavenumber'][:1864].values]], [['count']]) # rename column integers with wavenumber value
rgrid = rgrid.loc[:, :, 2:8, :] # Keep just lat's 2-8 (no arctic/antarctic)

if (rgrid.values <= 0).any():
    print(f'{round((rgrid[rgrid <= 0].count(axis=0).sum() / (rgrid.shape[0]*rgrid.shape[1]) * 100), 2)} percent of radiances are negative. Removing...')
    rgrid = rgrid[(rgrid > 0)]
    if (rgrid.values <= 0).any():
        print("Something didn't work, there are still negative radiances")

rgrid.dropna(axis = 1, how = 'all', inplace=True) # Remove the channels with negative (now NaN) radiances

year_count = rgrid.index.get_level_values(0).unique().values.max()-rgrid.index.get_level_values(0).unique().values.min() + 1
chan_count1 = len(rgrid.columns) - 1
spectra_count1 = rgrid['count'].sum()
radiance_count1 = spectra_count1 * chan_count1
print('Before filtering, there are an avg of', round(spectra_count1/1e6/year_count,2), 'million spectra per year')
print('Before filtering, there are', round(spectra_count1/1e6, 1), 'million spectra total')
print('Before filtering', round(radiance_count1/1e9, 1), 'billion radiances total')

# Filter by measurement count
if 1 == 1:
    before_count = len(rgrid)
    rgrid = rgrid.loc[(rgrid['count'] >= 5)]
    print(f'Removing {100*round((1-len(rgrid)/before_count), 3)}% of gridcells with <5 measurements/month')
    
# Remove channels with insufficient length of time
if 1 == 1:
    rcounts = pd.DataFrame(rgrid.groupby(['lat', 'lon']).count().sum(axis=0))
    keep_chan = rcounts.loc[rcounts[0] >= (rcounts[0].max()-0)] # can adjust record length inclusion criteria here.
    print('Keeping %s percent of channels with full record length' %round(len(keep_chan)/len(rcounts[0])*100, 2))
    rgrid = rgrid.loc[:, keep_chan.index.values]

spectra_count2 = rgrid['count'].sum()
rgrid = rgrid.drop(['count'], axis=1)
rgrid.columns = rgrid.columns.astype('float')
rgrid_ix = rgrid.drop(rgrid.columns, axis=1) # Define the AIRS index for intersecting with LBL later.
chan_count2 = len(rgrid.columns)
radiance_count2 = spectra_count2 * chan_count2
print('After filtering, there are an avg of', round(spectra_count2/1e6/year_count, 2), 'million spectra per year')
print('After filtering, there are', round(spectra_count2/1e6, 1), 'million spectra total')
print('After filtering', round(radiance_count2/1e9, 1), 'billion radiances total')
print('After filtering', spectra_count2*chan_count2/1E9/year_count, 'billion radiances per year, avg')
print('AIRS Memory: %s MB' %round(rgrid.memory_usage().sum()/2**20, 1))

In [ ]:
print(90 * 135 * 10 * 24 * 365 / 1e12 * chan_count2, 'trillion radiances/yr at 650-1620 cm-1')

In [ ]:
print(90 * 135 * 10 * 24 * 365 / 1e12 * 2378, 'trillion total radiances/yr (650-2600 cm-1)')

### Import LBL data:

In [ ]:
def lbl_import(file):
    lbl = pd.read_csv(file)
    lbl.set_index(['year', 'month', 'lat', 'lon'], inplace=True)
    lbl = lbl.loc[:, :, 2:8, :] # Keep just lat's 2-8 (no arctic/antarctic)
    lbl.columns = pd.to_numeric(lbl.columns, errors='coerce')
    print('Done. Memory use:', round(lbl.memory_usage().sum()/2**20, 1), 'MB')
    return lbl

In [ ]:
for file in glob.glob(folder_LBL_cfc+'*.gz'):
    print('Importing', file+"...")
    if 'ERA' in file:
        lcfc0 = lbl_import(file)
    if 'CFC11' in file:
        lcfc11 = lbl_import(file)
    if 'CFC12' in file:
        lcfc12 = lbl_import(file)

for file in glob.glob(folder_LBL_20x20+'*.gz'):
    print('Importing', file+"...")
    if 'ERA' in file:
        lgridd = lbl_import(file)
        lgridd = pd.merge(lgridd, rgrid_ix, how='inner', on=['year', 'month', 'lat', 'lon']) # note 1
        lgridd.columns = pd.to_numeric(lgridd.columns, errors='coerce')
    elif 'GHG' in file:
        lgriddg = lbl_import(file)
        lgriddg = pd.merge(lgriddg, rgrid_ix, how='inner', on=['year', 'month', 'lat', 'lon']) # note 1
        lgriddg.columns = pd.to_numeric(lgriddg.columns, errors='coerce')
    elif 'MET' in file:
        lgriddm = lbl_import(file)
        lgriddm = pd.merge(lgriddm, rgrid_ix, how='inner', on=['year', 'month', 'lat', 'lon']) # note 1
        lgriddm.columns = pd.to_numeric(lgriddm.columns, errors='coerce')
        
years = lgridd.index.get_level_values(0).unique().values.max()-lgridd.index.get_level_values(0).unique().values.min()

# Note 1: If LBL is generated where there is no AIRS data, the LBL results must be removed for these locations because
# they are spuriously high (10x) radiance where no L1b location was provided.

In [ ]:
# Check for timespan consistency between the two datasets
print('The LBL dataset does not have these years in AIRS:', 
      [x for x in rgrid.index.get_level_values(0).unique().values if x not in lgridd.index.get_level_values(0).unique().values]
     )
print('The AIRS dataset does not have these years in the LBL:', 
     [x for x in lgridd.index.get_level_values(0).unique().values if x not in rgrid.index.get_level_values(0).unique().values]
     )

# Common functions

In [ ]:
def mlma(df, wn):
    '''
    mlma = "Make LBL match AIRS" 
    Receives: LBL radiances and WN (the resolution index of AIRS radiances)
    Returns: LBL radiances spaced (indexed) exactly like AIRS, with result interpolated for each channel
    '''
    wnl = wn['wavenumber'].values.tolist()
    remove_list = (df.index & wn['wavenumber'].values).values.tolist()
    for r in remove_list:
        wnl.remove(r)
    df = pd.concat([df, pd.DataFrame(index=wnl)])
    df.sort_index(inplace = True)
    df.interpolate(inplace = True)
    #df.interpolate(method = 'nearest', inplace = True)
    #df.interpolate(method = 'quadratic', inplace = True)
    return df.loc[wn['wavenumber'].values]

def linr(df):
    '''
    Receives: dataframe of annual radiances at various wavenumbers
    Returns: dataframe of slopes from linear regression over time for each wavenumber
    '''
    # For upcoming loops, need lists of wavenumbers and years
    wns = df.index.get_level_values(1).unique().values
    years = df.xs(wns[0], axis=0, level=1).T.columns.values

    # Linear regression (slopes) for each wavenumber in each lat band
    sl = pd.DataFrame(index = wns, columns = df.columns.values) # sl == slopes
    for wn in wns:
        clear_output(wait=True); print("Linear regression:", round(wn,0), "of", wns.max())
        for lat, row in df.xs(wn, axis=0, level=1).T.iterrows():
            sl.loc[wn].at[lat], _, _, _, _ = scipy.stats.linregress(years, row.values)
    clear_output(wait=True); print("Finished.")
    return sl.astype(float)

## Figure 1 - Multipanel

In [ ]:
from pyhdf.SD import *

def square(x, y, width, color, a):
    from matplotlib.patches import Rectangle
    j = matplotlib.colors.to_rgb(color)
    #from matplotlib.patheffects import withStroke
    sq = Rectangle((x, y), width, width, clip_on=False, zorder=10, linewidth=0.4, 
                    edgecolor=(j[0], j[1], j[2], a), facecolor=(0, 0, 0, 0))
                    #path_effects=[withStroke(linewidth=1, foreground='w')])
    return sq
    
def ir_at(y, x):
    OLR = pd.DataFrame(list(zip(wn['wavenumber'], ir[y,x,:])), columns=['wavenumber', 'radiance'])
    #OLR = OLR.round(3)
    OLR = OLR.loc[OLR.radiance > 0.1]
    OLR.set_index('wavenumber', inplace = True)
    return OLR

def image(rads):
    YTrack = len(rads[:,0,0,0,0])
    XTrack = len(rads[0,:,0,0,0])
    SubTrack = len(rads[0,0,0,:,0])
    SubXTrack = len(rads[0,0,0,0,:])
    delta_y = 8
    colors = [0, 1, 2]
    df2 = {}

    for color in colors:
        df = pd.DataFrame()

        for y in np.arange(0,135,1):
            y = int(y)
            dfmid = pd.DataFrame()

            for x in np.arange(0, 90, 1):
                data_in = pd.DataFrame(data=rads[y,int(x),color,:delta_y,:], 
                                       index=np.arange(y*delta_y, (1+y)*delta_y, 1), 
                                       columns = np.arange(x*SubXTrack, (1+x)*SubXTrack, 1)
                                      )
                dfmid = pd.concat([dfmid, data_in], axis=1)

            df = df.append(dfmid) 

        df = df/df.max().max() # Normalize to 1 for color assignment (0-1 RGB value)
        df2[color] = df.sort_index().sort_index(axis=1)

    df = np.dstack([df2[2]**0.85,df2[1]**0.85,df2[0]**0.85]) #dstack R, G, B dataframes
    return df

In [ ]:
jumps = [-1]
for i in range(len(wn['wavenumber']) - 1):
    if wn['wavenumber'][i+1] - wn['wavenumber'][i] > 15:
        jumps.append(i)
jumps.append(len(wn['wavenumber'])-1)
jumps

In [ ]:
f = SD(AIRS_vis)
rads = f.select('radiances')
lats = f.select('cornerlats')
lons = f.select('cornerlons')
dfvis = image(rads)

f = SD(AIRS_ir)
ir = f.select('radiances')
cld = f.select('spectral_clear_indicator')

clrx = []
clry = []
for y in range(cld[:,:].shape[0]):
    for x in range(cld[:,:].shape[1]):
        if cld[y,x] == -2:
            clrx.append(x)
            clry.append(y)
        elif cld[y,x] == 2:
            clrx.append(x)
            clry.append(y)

In [ ]:
j = 4

width = dfvis.shape[1]
height = dfvis.shape[0]

clrir = ir_at(clry[j], clrx[j])
fig, ax = plt.subplots(ncols=3, figsize=(14, 6))

ax[0].imshow(dfvis, aspect = 1.0, origin = 'lower')
ax[0].set_xticks([])
ax[0].set_yticks([])

for j in range(len(clrx)):
    if ((clrx[j] >= 32) & (clrx[j] <= 59)):
        ax[0].add_artist(square(clrx[j]/90*width, clry[j]/135*height, width/110, 'yellow', 1.0))
    else:
        ax[0].add_artist(square(clrx[j]/90*width, clry[j]/135*height, width/110, 'yellowgreen', 0.9))

for n in range(len(jumps)-3):
    clrir.loc[slice(wn['wavenumber'][jumps[n]+1], wn['wavenumber'][jumps[n+1]])].plot(lw=0.15, c='k', ax=ax[1])
ax[1].set_ylim([0, 165])
ax[1].set_xlim([600, 1650])
ax[1].minorticks_on()
ax[1].legend().set_visible(False)
ax[1].set_ylabel(r'Radiance, mW$\,$m$^{-2}$(cm$^{-1}$)$^{-1}$sr$^{-1}$', fontsize=10)
ax[1].set_xlabel(r'Wavenumber, cm$^{-1}$', fontsize=10)

clrir.loc[slice(wn['wavenumber'][0], wn['wavenumber'][129])].plot(lw=0.4, c='k', ax=ax[2])
clrir.loc[slice(wn['wavenumber'][130], wn['wavenumber'][210])].plot(lw=0.4, c='k', ax=ax[2])
ax[2].set_ylim([-20, 80])
ax[2].set_yticks([30, 40, 50, 60, 70, 80])
ax[2].tick_params(axis='both', direction='out', which='both')
ax[2].set_xlim([630, 710])
ax[2].legend().set_visible(False)
ax[2].set_ylabel(r'Radiance, mW$\,$m$^{-2}$(cm$^{-1}$)$^{-1}$sr$^{-1}$', ha='left', fontsize=10)
ax[2].set_xlabel(r'Wavenumber, cm$^{-1}$', fontsize=10)

ax3 = ax[2].twinx()
markerline, stemlines, baseline = ax3.stem(HITRAN2020['Wavenumber'], 
                                           HITRAN2020['Absorbance'], 
                                           markerfmt=' ', linefmt="g-", basefmt="g-", use_line_collection=True
                                          )
plt.setp(stemlines, 'linewidth', 0.4)

ax3.set_ylim([0, 6])
ax3.set_yticks([0, 1, 2, 3])
ax3.minorticks_on()
ax3.yaxis.set_tick_params(which='minor', right=False)
ax3.set_ylabel('HITRAN Absorbance\nx10$^{-19}$ cm$^{-1}$ mol$^{-1}$ cm$^2$', ha='right', fontsize=11, color='g')
ax3.tick_params(axis='both', direction='out', which='both')

if 1 == 2:
    plt.savefig('.\fig1.png')

plt.show()

## Figure 2

In [ ]:
# Calculate global average radiance, global change
# Weights for computing one global average radiance:
weights = []
for lat in range(7):
    weights.append(2 * np.pi * (np.sin(np.radians(70-lat*20)) - (np.sin(np.radians(70-(lat+1)*20)))) / 4 / np.pi)
weights = weights / sum(weights)
assert sum(weights) == 1.0, "weights don't sum to 1.0"

In [ ]:
# Global average radiance
globalr = (rgrid.groupby(['lat']).mean().T * weights).T.sum()
globalr[globalr == 0] = np.nan # Where radiance = 0, replace with nan to avoid plotting
globall = (lgridd.groupby(['lat']).mean().T * weights).T.sum()

# Linregress
# Each lon segment in a given lat is equal area, so mean() across lon's weights them evenly
# Months are all averaged into a single year before linregress because intra-year seasonality can create spurious trends
dfr = linr(rgrid.groupby(['year', 'lat', 'lon']).mean().groupby(['year', 'lat']).mean()) # avg the lon's and month's, then linregress.
dfl = linr(lgridd.groupby(['year', 'lat', 'lon']).mean().groupby(['year', 'lat']).mean()) 
dflg = linr(lgriddg.groupby(['year', 'lat', 'lon']).mean().groupby(['year', 'lat']).mean())
dflm = linr(lgriddm.groupby(['year', 'lat', 'lon']).mean().groupby(['year', 'lat']).mean())

In [ ]:
# Prepare data for joint grids 1 & 2

# Warning: 19 years uses 4+ GB memory and can crash as non-addressable. 
# This is why it converts float64 to float32 (saves ~15% mem)
yr1 = 2003
yr2 = 2021
wn1 = 649.0
wn2 = 1650.0

ljoint = lgridd.loc[:, wn1:wn2].loc[yr1:yr2].astype('float32')
ljoint = mlma(ljoint.T, wn[(wn['wavenumber'] >=wn1) & (wn['wavenumber'] < wn2)]).T.astype('float32')
rjoint = rgrid.loc[:, wn1:wn2].loc[yr1:yr2].astype('float32')
for i in rjoint.columns.values:
    rjoint.loc[(rjoint[i] <= 0), i] = np.nan
rjoint = rjoint.rename_axis('wavenumber', axis='columns')

jg1 = pd.DataFrame(ljoint.stack(), columns = ['LBL']).rename_axis(['year', 'month', 'lat', 'lon', 'wavenumber']).join(
    pd.DataFrame(rjoint.stack(dropna = False), columns = ['AIRS']), 
    how='left')

ljoint = dfl.loc[:, wn1:wn2]
ljoint = mlma(ljoint.T, wn[(wn['wavenumber'] >=wn1) & (wn['wavenumber'] < wn2)]).T
rjoint = dfr.loc[:, wn1:wn2]
for i in rjoint.columns.values:  # Remove a handful of extreme outliers, usually due to incomplete timeseries
    rjoint.loc[(rjoint[i] > 1.75), i] = np.nan
    rjoint.loc[(rjoint[i] < -1.75), i] = np.nan

jg2 = pd.DataFrame(ljoint.stack(), columns = ['LBL']).rename_axis(['lat', 'wavenumber']).join(
    pd.DataFrame(rjoint.stack(dropna = False), columns = ['AIRS']).rename_axis(['lat', 'wavenumber']), 
    how='left')
jg2 = jg2.dropna()

In [ ]:
def jgp(df, save, savename):
    sns.set_theme(style="ticks")
    
    fig = plt.figure(figsize=(2, 1.5))    
    g = sns.JointGrid(data=df, x="AIRS", y="LBL", marginal_ticks=True)

    # Create an inset legend for the histogram colorbar
    cax = g.fig.add_axes([.15, .55, .02, .2])

    # Add the joint and marginal histogram plots
    g.plot_joint(
        sns.histplot, #discrete=(True, False),
        cmap="magma_r", pmax = .9, cbar=True, cbar_ax=cax  # cmap="light:#03012d"
    )
    
    # Plot y=x line
    ax = g.ax_joint
    ax.plot(ax.get_xlim(), ax.get_xlim(), ls=":", lw=1, alpha=0.6, label="1:1")
    if savename == 'f':
        delta = ' rate of change'
        delta1 = 'yr$^{-1}$'
    else:
        delta = ''
        delta1 = ''
    ax.set_xlabel('AIRS Radiance'+delta+', mW$\,$m$^{-2}$(cm$^{-1}$)$^{-1}$sr$^{-1}$'+delta1)
    ax.set_ylabel('LBL Radiance'+delta+', mW$\,$m$^{-2}$(cm$^{-1}$)$^{-1}$sr$^{-1}$'+delta1)
                
    g.plot_marginals(sns.histplot, element="step", color="#03012d")
    
    # Histograms acquire different max values, making their apparent shapes slightly different.
    # Intervene and force the same y-max on both:
    marg_max = max(g.ax_marg_x.get_ylim()[1], g.ax_marg_y.get_xlim()[1])
    ax = g.ax_marg_x
    ax.set_ylim(0, marg_max)
    ax = g.ax_marg_y
    ax.set_xlim(0, marg_max)
        
    plt.figtext(0.9, .9, savename, ha = 'right', fontsize = 30)
    
    if save:
        g.savefig('.\fig2'+savename+'.png')
    plt.show()
    return

In [ ]:
print('There are', round((len(jg1)/1E6), 1), 'million monthly avg radiances in jg1')
print('There are', round((len(jg2)/1E3), 1), 'thousand monthly avg radiances in jg2')

In [ ]:
jgp(jg1, False, 'e')

In [ ]:
jgp(jg2, False, 'f')

In [ ]:
fig = plt.figure(figsize=(10, 16)) #constrained_layout=True,
num = 6
ax = [None]*(num + 1)

gs = GridSpec(8, 11)
ax[0] = fig.add_subplot(gs[0, :])
ax[1] = fig.add_subplot(gs[1, :])
ax[2] = fig.add_subplot(gs[2, :])
ax[3] = fig.add_subplot(gs[3, :])
ax[4] = fig.add_subplot(gs[3:, :5])
ax[5] = fig.add_subplot(gs[3:, 6:])

ax[0].plot(globalr[:1136.634].index, globalr[:1136.634].values, color = 'black')
ax[0].plot(globalr[1137:].index, globalr[1137:].values, color = 'black', label = 'AIRS')
ax[0].plot(globall.index, globall.values, color = 'blue', label = 'LBL')

leg = ax[0].legend()

ax[1].plot(dfr.T.index[:130], (dfr.T * weights).T.sum().values[:130], color = 'black',  label = 'AIRS')
ax[1].plot(dfr.T.index[131:1083], (dfr.T * weights).T.sum().values[131:1083], color = 'black', label = 'AIRS')
ax[1].plot(dfr.T.index[1084:], (dfr.T * weights).T.sum().values[1084:], color = 'black', label = '_')
ax[1].plot(dfl.T.index, (dfl.T * weights).T.sum().values, color = 'blue', label = 'LBL')
ax[2].plot((dflm.T*weights).T.sum().index, (dflm.T*weights).T.sum().values, color = 'blue')
ax[3].plot((dflg.T*weights).T.sum().index, (dflg.T*weights).T.sum().values, color = 'blue')

ax[0].set_ylabel(r'Radiance, mW$\,$m$^{-2}$(cm$^{-1}$)$^{-1}$sr$^{-1}$')
ax[2].set_ylabel(r'Radiance rate of change, mW$\,$m$^{-2}$(cm$^{-1}$)$^{-1}$sr$^{-1}$yr$^{-1}$')
ax[3].set_xlabel(r'Wavenumber, cm$^{-1}$')

for n in np.arange(1, 4, 1):
    ax[n].axhline(0, color='black', linestyle='--')

img1 = plt.imread("fig2e.png")
ax[4].imshow(img1)
ax[4].axis('off')

img2 = plt.imread("fig2f.png")
ax[5].imshow(img2)
ax[5].axis('off')

if 1 == 2:
    plt.savefig('fig2.png')
plt.show()

# Figure 3

In [ ]:
lats = ['70°S to 50°S', '50°S to 30°S', '30°S to 10°S', '10°S to 10°N', 
        '10°N to 30°N', '30°N to 50°N', '50°N to 70°N']

In [ ]:
num = 7
fig = plt.figure()
gs = gridspec.GridSpec(nrows=num, ncols=1, figure=fig, height_ratios= [1]*num)
ax = [None]*(num + 1)

for i in range(num):
    n = num-i+1
    ax[i] = fig.add_subplot(gs[i])
    ax[i].axhline(0, color='black')
    
    ax[i].plot(dfl.loc[n].index, dfl.loc[n].values, color = 'blue', label = 'LBL')
    ax[i].plot(dfr.loc[n].index[:1083], dfr.loc[n].values[:1083], color = 'black', label = 'AIRS')
    ax[i].plot(dfr.loc[n].index[1084:], dfr.loc[n].values[1084:], color = 'black', label = 'AIRS')
    ax[i].annotate(lats[n-2], (1450, -0.1), color='black')

ax[3].set_ylabel(r'Radiance rate of change, mW$\,$m$^{-2}$(cm$^{-1}$)$^{-1}$sr$^{-1}$yr$^{-1}$')
ax[i].set_xlabel(r'Wavenumber, cm$^{-1}$')

if 1 == 2:
    plt.savefig('fig3.png')
plt.show()

# Figure 4 Histogram

In [ ]:
c1 = 710 # cm-1, lower bound
c2 = 720 # cm-1, upper bound
path_compare = pd.DataFrame()

#### Prepare Fig 4 AIRS histogram data

In [ ]:
p5 = pd.DataFrame()
for month in np.arange(1, 13, 1):
    for lon in np.arange(1, 19, 1):
        try:
            test = linr(rgrid.loc[:, c1:c2].loc[:, month, :, lon]).rename_axis(['lat'])
            test['lon'] = lon
            test['month'] = month
            test.set_index(['lon', 'month'], append=True, inplace = True)
            p5 = p5.append(test)
        except:
            pass
p5.sort_index(inplace = True)
p5o = pd.DataFrame(index = p5.index, columns = ['AIRS'], dtype='float')

for row in p5.iterrows():
    p5o.loc[row[0][0], row[0][1], row[0][2]]['AIRS'] = scipy.integrate.trapz(row[1].values, row[1].index.values)
p5om = p5o.groupby(['lat']).mean()
path_compare = pd.concat([path_compare, p5om], axis=1)
path_compare.rename_axis(['lat'], inplace=True)

#### Prepare LBL Fig. 4 histogram Data

In [ ]:
p5l = pd.DataFrame()
for month in np.arange(1, 13, 1):
    for lon in np.arange(1, 19, 1):
        try:
            test = linr(lgridd.loc[:, c1:c2].loc[:, month, :, lon]).rename_axis(['lat'])
            test['lon'] = lon
            test['month'] = month
            test.set_index(['lon', 'month'], append=True, inplace = True)
            p5l = p5l.append(test)
        except:
            pass
p5l.sort_index(inplace = True)
p5lo = pd.DataFrame(index = p5l.index, columns = ['LBL'], dtype='float')

for row in p5l.iterrows():
    p5lo.loc[row[0][0], row[0][1], row[0][2]]['LBL'] = scipy.integrate.trapz(row[1].values, row[1].index.values)

In [ ]:
p5hist = pd.concat([p5o*years, p5lo*years], axis=1)
palette = {"AIRS":"tab:grey",
           "LBL":"tab:blue"}
fig, ax = plt.subplots(figsize=(6, 5))
ax = sns.histplot(data=p5hist.melt(), x="value", hue="variable", element="step", palette = palette, stat='density')
ax.set_xlabel('Radiant flux [mW$\,$m$^{-2}$sr$^{-1}$]')
ax.axvline((p5hist.groupby(['lat', 'month']).mean().groupby(['lat']).mean().T* weights).sum(axis=1)['AIRS'], color='k', alpha=0.4, lw=2)
ax.axvline((p5hist.groupby(['lat', 'month']).mean().groupby(['lat']).mean().T* weights).sum(axis=1)['LBL'], color='b', alpha = 0.4, lw=2)
if 1 == 2:
    plt.savefig('fig4b.png')
plt.show()

# Other pathways (not used, preservation only)

## Pathway 1

In [ ]:
p1o = pd.DataFrame(index = np.arange(2, 9, 1), columns = ['Pathway 1'])
p1 = linr(rgrid.loc[:, c1:c2].groupby(['year', 'lat', 'lon']).mean().groupby(['year', 'lat']).mean())
for lat in np.arange(2, 9, 1):
    p1o.loc[lat]['Pathway 1'] = scipy.integrate.trapz(p1.loc[lat].values, p1.loc[lat].index.values)
path_compare = pd.concat([path_compare, p1o], axis=1)

## Pathway 2

In [ ]:
p2o = pd.DataFrame(index = np.arange(2, 9, 1), columns = ['Pathway 2'])
p2 = pd.DataFrame()
for lon in np.arange(1, 19, 1):
    test = linr(rgrid.loc[:, c1:c2].groupby(['year', 'lat', 'lon']).mean().loc[:, :, lon]).rename_axis(['lat'])
    #test = test.rename_axis(['lat'])
    test['lon'] = lon
    test.set_index(['lon'], append=True, inplace = True)
    p2 = p2.append(test)
p2 = p2.groupby(['lat']).mean()
for lat in np.arange(2, 9, 1):
    p2o.loc[lat]['Pathway 2'] = scipy.integrate.trapz(p2.loc[lat].values, p2.loc[lat].index.values)
path_compare = pd.concat([path_compare, p2o], axis=1)

## Pathway 3

In [ ]:
p3o = pd.DataFrame(index = np.arange(2, 9, 1), columns = ['Pathway 3'])
p3 = pd.DataFrame()
for month in tqdm(np.arange(1, 13, 1)):
    for lon in np.arange(1, 19, 1):
        try:
            test = linr(rgrid.loc[:, c1:c2].loc[:, month, :, lon]).rename_axis(['lat'])
            test['lon'] = lon
            test['month'] = month
            test.set_index(['lon', 'month'], append=True, inplace = True)
            p3 = p3.append(test)
        except:
            pass
p3.sort_index(inplace = True)
p3 = p3.groupby(['lat', 'lon']).mean().groupby(['lat']).mean()
for lat in np.arange(2, 9, 1):
    p3o.loc[lat]['Pathway 3'] = scipy.integrate.trapz(p3.loc[lat].values, p3.loc[lat].index.values)
path_compare = pd.concat([path_compare, p3o], axis=1)

## Pathway 4

In [ ]:
p4o = pd.DataFrame(index = np.arange(2, 9, 1), columns = ['Pathway 4'])
p4 = pd.DataFrame()
for month in tqdm(np.arange(1, 13, 1)):
    for lon in np.arange(1, 19, 1):
        try:
            test = linr(rgrid.loc[:, c1:c2].loc[:, month, :, lon]).rename_axis(['lat'])
            test['lon'] = lon
            test['month'] = month
            test.set_index(['lon', 'month'], append=True, inplace = True)
            p4 = p4.append(test)
        except:
            pass
p4.sort_index(inplace = True)
p4 = p4.groupby(['lat', 'month']).mean().groupby(['lat']).mean()
for lat in np.arange(2, 9, 1):
    p4o.loc[lat]['Pathway 4'] = scipy.integrate.trapz(p4.loc[lat].values, p4.loc[lat].index.values)
path_compare = pd.concat([path_compare, p4o], axis=1)

In [ ]:
intgrls = {}

intgrls['Observed'] = scipy.integrate.trapz((dfr.loc[:, c1:c2].T*weights).T.sum().values, dfr.loc[:, c1:c2].T.index)*years
intgrls['Simulated'] = scipy.integrate.trapz((dfl.loc[:, c1:c2].T*weights).T.sum().values, dfl.loc[:, c1:c2].T.index)*years
intgrls

# Supplementary

## Fig S4

In [ ]:
cfc11 = (lcfc11.groupby(['lat', 'lon']).mean().groupby(['lat']).mean().T * weights).T.sum() - (lcfc0.groupby(['lat', 'lon']).mean().groupby(['lat']).mean().T * weights).T.sum().loc[:1600]
cfc12 = (lcfc12.groupby(['lat', 'lon']).mean().groupby(['lat']).mean().T * weights).T.sum() - (lcfc0.groupby(['lat', 'lon']).mean().groupby(['lat']).mean().T * weights).T.sum().loc[:1600]

In [ ]:
num = 3
fig = plt.figure(figsize=(8, 8))
gs = gridspec.GridSpec(nrows=num, ncols=1, figure=fig, height_ratios= [1]*num)
ax = [None]*(num + 1)

for n in range(num):
    ax[n] = fig.add_subplot(gs[n])
    ax[n].set_xlim(780, 999)
    ax[n].set_ylim(-0.045, 0.055)
    ax[n].axhline(0, color='black')

ax[0].plot(dfr.T.index, (dfr.T * weights).T.sum().values, color = 'black', label = 'AIRS')
ax[0].plot(dfl.T.index, (dfl.T * weights).T.sum().values, color = 'blue', label = 'LBL')

ax[1].plot((dflg.T*weights).T.sum().index, (dflg.T*weights).T.sum().values, color = 'blue')

ax[1].set_ylabel(r'Radiance rate of change, mW$\,$m$^{-2}$(cm$^{-1}$)$^{-1}$sr$^{-1}$yr$^{-1}$')
ax[2].set_xlabel(r'Wavenumber, cm$^{-1}$')

ax[2].plot(cfc11.index, cfc11.values/years, lw=0.75, label='CFC-11', color = 'red')
ax[2].plot(cfc12.index, cfc12.values/years, lw=0.75, label='CFC-12', color = 'green')
ax[2].legend()

if 1 == 2:
    plt.savefig('FIG-S4.png')
plt.show()

## Fig S5

In [ ]:
rloc = pd.DataFrame()
for year in np.arange(2003, 2022, 1):
    rloci = pd.DataFrame()
    files = glob.glob('C:\\data\\location\\'+str(year)+'\\*.csv')
    if len(files) != 12:
        print(len(files), 'files found, instead of 12')
    for file in files:
        rloci = rloci.append(pd.read_csv(file))
    try:
        rloci = rloci.drop(['sci'], axis=1)
    except:
        pass
    rloci = rloci.set_index(['time'])
    rloci = rloci.astype('float16')
    rloc = rloc.append(rloci)
    print(f'Year {year} done. Memory use:', round(rloc.memory_usage().sum()/2**20, 1), 'MB')

rloci = []
if rloc.isnull().values.any():
    print('nan detected in final rloc dataframe')

In [ ]:
import cartopy.crs as ccrs
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)

xedges = np.linspace(-180,180,181)
yedges = np.linspace(-90,90,91)

fig = plt.figure(figsize=(18, 20), constrained_layout=True)
gs0 = fig.add_gridspec(7, 3)

for a in range(7):
    for b in range(3):
        year = 2003 + 3 * a + b
        if year > 2021:
            break
        print("processing "+str(year)+"...")
        ax = fig.add_subplot(gs0[a, b], projection=ccrs.EqualEarth())
        ax.coastlines()
        ax.set_global()
        rloc2 = rloc.loc[str(year)+'-01-01':str(year+1)+'-01-01']
        x = rloc2['lon'].values
        y = rloc2['lat'].values
        H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
        H = H.T  # Let each row list bins with common y range.
        cf = ax.pcolormesh(xedges, yedges, H, transform=ccrs.PlateCarree(), vmax = 2000, cmap='magma_r') 
        ax.set_title(str(year))
        gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.25, color='black', alpha=0.5)
        gl.ylocator = mticker.FixedLocator(np.arange(-90, 91, 20))
        gl.xlocator = mticker.FixedLocator(np.arange(-180, 181, 20))
        gl.xformatter = LongitudeFormatter()
        gl.yformatter = LatitudeFormatter()
ax = fig.add_subplot(gs0[6, 1])
ax.axis('off')
ax.annotate('Count', (0.4, 0.11), color='black', fontsize=18)
fig.colorbar(cf, ax=ax, location='bottom')
if 1 == 2:
    plt.savefig('FIG-S5.png')
plt.show()

## Is the result here consistent with 0.53 W/m2?

In [ ]:
df = (dflg.T*weights).T.sum()
check = scipy.integrate.trapz(df[:1000].values, df[:1000].index.values) * 19 * np.pi / 1000
print('The LBL integral for 0.1 to 1000 cm-1 is', round(check, 3), 'W/m2, which is', round((check/-0.53*100), 1), 'percent of 0.53 W/m2')